In [ ]:
from datasets import load_dataset
import tiktoken
from pathlib import Path
import os
import torch
import random
import pprint
import tiktoken

In [2]:
ds = load_dataset("HuggingFaceTB/smol-smoltalk", split="train")

Generating test split: 100%|██████████| 24229/24229 [00:00<00:00, 436558.32 examples/s]


In [67]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')

buffer=[]
print(len(ds))
output_dir=Path("../data/smol-smoltalk")
file_idx=0
token_count=0
max_tokens = 10_240_000

for i in range(10):
    # Step 1: flatten all messages into one long token list per dataset item
    all_tokens = []
    for message in ds[i]['messages']:
        role = message['role']
        content = message['content']

        if role == 'user':
            text = "<s>" + content + "</s>"
        else:
            text = "<s>" + content + "</s>"

        tokens = enc.encode(text)
        len_token=len(tokens)

        if token_count+len_token>=max_tokens:
            output_path = output_dir / f"tokens_part_{file_idx}.pt"
            torch.save(torch.tensor(buffer, dtype=torch.long), output_path)

            file_idx += 1
            buffer = []
            token_count = 0

        buffer.extend(tokens)
        token_count+=len_token

if buffer:
    output_path = output_dir / f"tokens_part_{file_idx}.pt"
    torch.save(torch.tensor(buffer, dtype=torch.long), output_path)
    


        

        

460341


In [64]:
from torch.utils.data import IterableDataset, DataLoader, DistributedSampler

class DirStreamingDataset(IterableDataset):
    """
    openwebtext directory now contains pt files in binary, precomputed with the GPT 2 encodings

    Each file is 10 million tokens, total of 904 files
    """

    def __init__(self, path: str | os.PathLike, block_size: int):
        self.dir_path = Path(path)
        if not self.dir_path.exists():
            raise FileNotFoundError(path)
        
        self.token_files=sorted(self.dir_path.glob("*.pt"))
        if not self.token_files:
            raise ValueError(f"No .pt files found in {self.dir_path}")
        self.num_files=len(self.token_files)
        self.block_size = block_size
        self.num_token_per_file=10000000

        # introduce per-worker offset to avoid every worker seeing the
        # same sequence order when shuffling is disallowed.
        # DataLoader(worker_init_fn) will set this value.
        self._worker_offset = 0


    # DataLoader with an IterableDataset must NOT shuffle. We instead start each worker at a random offset to approximate randomness.
    def set_worker_offset(self, offset: int):
        self._worker_offset = offset

    def get_dataset_token_count(self):
        return self.num_files*self.num_token_per_file

    def __iter__(self):
        rng = torch.Generator().manual_seed(42 + self._worker_offset)
        file_indices = list(range(len(self.token_files)))
        random.shuffle(file_indices)

        while True:
            for idx in file_indices:
                tokens = torch.load(self.token_files[idx])
                n = len(tokens)
                pos = torch.randint(0, n - self.block_size - 1, (1,), generator=rng).item()

                while pos + self.block_size + 1 < n:
                    #now instead of randomising the start position, we randomise which file we see, then the start position is sequential
                    chunk = tokens[pos : pos + self.block_size + 1]
                    x = chunk[:-1]
                    y = chunk[1:]
                    pos += self.block_size
                    yield x, y# DataLoader → (B,T)

def make_loader(dataset: DirStreamingDataset, batch_size: int, num_workers: int,
                sampler: DistributedSampler | None, device_is_cuda: bool):
    def worker_init_fn(worker_id: int):
        worker_info = torch.utils.data.get_worker_info()
        dataset: DirStreamingDataset = worker_info.dataset
        offset = random.randint(0, len(dataset.tokens) - 1)
        dataset.set_worker_offset(offset)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        sampler=sampler,
        num_workers=num_workers,
        pin_memory=device_is_cuda,
        prefetch_factor=None, # does not run when 0 num_workers
        persistent_workers=(num_workers > 0),
        worker_init_fn=None,
    )

In [94]:
data_path="../data/smol-smoltalk"
B, T        = 64, 1024
lr          = 3e-4
max_steps   = 10
enc         = tiktoken.get_encoding("gpt2")
sampler=None

dataset = DirStreamingDataset(path=data_path, block_size=T)
loader = make_loader(
        dataset,
        batch_size=B,
        num_workers=0,            # no separate workers ➜ no pickling
        sampler=sampler,
        device_is_cuda=False,
    )

for step, (x, y) in enumerate(loader):
    if step >= max_steps:
        break

    print(enc.decode(x[0].tolist()))
    print(x)
    print("---")
    print(y)

 dispenser 10 or another piece of sheet material 12 contained therein. The dispenser 10 is provided with a dispensing mechanism 36 that allows for a piece of sheet material 12 to be dispensed from the dispenser 10 by a user and subsequently automatically advances a second piece of sheet material 12 for future removal so that a user does not have to touch the dispenser 10. The dispensing mechanism 36 may be configured in a number of ways in accordance with various exemplary embodiments of the present invention. For instance, the dispensing mechanism 36 may be configured so that the user will draw the sheet material 12 against a stationary cutting blade 22 in order to sever the piece of sheet material 12 and cause removal. Alternatively, the dispensing mechanism 36 may be configured so that the cutting blade 22 is moveable so as to automatically sever the sheet material 12 when pulled by a user.
FIGS. 1 and 2 show the dispenser 10 with a dispensing mechanism 36 that is in an at rest or r